In [ ]:
import numpy as np
import pandas as pd
import os

!mkdir -p drive
!google-drive-ocamlfuse drive
#os.chdir("drive")
data_dir = "drive/demo/"

'''
!pip install torch==1.4.0 torchvision==0.5.0

!pip install torch-geometric \
  torch-sparse==latest+cu101 \
  torch-scatter==latest+cu101 \
  torch-cluster==latest+cu101 \
  -f https://pytorch-geometric.com/whl/torch-1.4.0.html
'''

import torch
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

import random
def fix_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
fix_seed(1234)


class GCN(torch.nn.Module):

    def __init__(self, num_layers=3, hidden=32,  features_num=16, num_class=2):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(features_num, hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(num_layers - 1):
            self.convs.append(GCNConv(hidden, hidden))
        self.lin2 = Linear(hidden, num_class)
        self.first_lin = Linear(features_num, hidden)

    def reset_parameters(self):
        self.first_lin.reset_parameters()
        self.conv1.reset_parameters()
        for conv in self.convs:
            conv.reset_parameters()
        self.lin2.reset_parameters()

    def forward(self, data):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_weight
        x = F.relu(self.first_lin(x))
        x = F.dropout(x, p=0.8, training=self.training)
        for conv in self.convs:
            x = F.relu(conv(x, edge_index, edge_weight=edge_weight))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)

    def __repr__(self):
        return self.__class__.__name__

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

def generate_data():
  x = pd.read_csv(data_dir + 'feature.tsv', sep='\t', dtype=float)
  x = x.drop('node_index', axis=1).to_numpy()

  x = torch.tensor(x, dtype=torch.float)

  edge_df = pd.read_csv(data_dir + 'edge.tsv', sep='\t', dtype=int)
  edge_index = edge_df[['src_idx', 'dst_idx']].to_numpy()
  edge_index = sorted(edge_index, key=lambda d: d[0])
  edge_index = torch.tensor(edge_index, dtype=torch.long).transpose(0, 1)

  edge_weight = edge_df['edge_weight'].to_numpy()
  edge_weight = torch.tensor(edge_weight, dtype=torch.float32)

  num_nodes = x.size(0)
  y = torch.zeros(num_nodes, dtype=torch.long)
  train_label = pd.read_csv(data_dir + 'train_label.tsv', sep='\t', dtype=int)
  inds = train_label[['node_index']].to_numpy()
  train_y = train_label[['label']].to_numpy()
  y[inds] = torch.tensor(train_y, dtype=torch.long)

  test_label = pd.read_csv(data_dir + 'test_label.tsv', sep='\t', dtype=int)
  inds = test_label[['node_index']].to_numpy()
  test_y = test_label[['label']].to_numpy()
  y[inds] = torch.tensor(test_y, dtype=torch.long)

  def read_txt(data_dir):
    output = []
    with open(data_dir, 'r') as f:
      line = f.readline()
      while line != '':
        output.append(line.strip())
        line = f.readline()
    f.close()
    return np.array(output, dtype=np.int32)
  train_indices = read_txt(data_dir + "train_node_id.txt")
  test_indices = read_txt(data_dir + "test_node_id.txt")

  data = Data(x=x, edge_index=edge_index, y=y, edge_weight=edge_weight)

  data.num_nodes = num_nodes

  train_mask = torch.zeros(num_nodes, dtype=torch.bool)
  train_mask[train_indices] = 1
  data.train_mask = train_mask

  test_mask = torch.zeros(num_nodes, dtype=torch.bool)
  test_mask[test_indices] = 1
  data.test_mask = test_mask

  print(num_nodes, edge_weight.size(0), data.y.max(0), data.x.size(1))

  return data

def train(model, data, optimizer, epoch):
  model.train()
  optimizer.zero_grad()
  loss = F.nll_loss(model(data)[data.train_mask], data.y[data.train_mask])
  loss.backward()
  optimizer.step()
  print("Epoch {} Train_loss: {}".format(epoch, loss.item()))
  
  return model

def test(model, data, epoch):
  model.eval()
  data = data.to(device)
  with torch.no_grad():
    prob = model(data)[data.test_mask]
    pred = prob.max(1)[1]
    loss = F.nll_loss(prob, data.y[data.test_mask])

  pred = pred.cpu()
  acc = data.y[data.test_mask].eq(pred).sum().float() / pred.size(0)
  print("Epoch {} Test_loss: {} Accuracy: {}".format(epoch, loss.item(), acc))

def main():
  data = generate_data()
  model = GCN(features_num=data.x.size()[1], num_class=int(max(data.y)) + 1)
  model = model.to(device)
  data = data.to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

  for epoch in range(100):
    model = train(model, data, optimizer, epoch)
    test(model, data, epoch)

main()

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


FileNotFoundError: ignored

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [1]:
import torch
import numpy as np
import random
import time
from datetime import datetime

a = np.array([1,2,3])
b = np.array([3,4, 5])
c = a * b
print(c)

[ 3  8 15]
